In [1]:
## script to calculate PA and OECMcoverage of terrestrial ecoregions

In [2]:
## stop outputs being added to the map
arcpy.env.addOutputsToMap = False

In [3]:
# read in ecorgions and wdpa
## these should be projected to Mollweide and geom repaired
eco = 'O:/PP2024/Ecoregions2017.gdb/Ecoregions2017_moll'
wdpa24 = 'O:/PP2024/wdpa_Aug2024_v2.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2024_Aug2024_PA_DEF_moll'
wdpa22 = 'O:/PP2024/wdpa_Aug2024_v2.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2022_Aug2024_PA_DEF_moll'
wdpa20 = 'O:/PP2024/wdpa_Aug2024_v2.gdb/wdpa_poly_point_public_restricted_basemap_intersect_statusyr2020_Aug2024_PA_DEF_moll'
oecm24 = 'O:/PP2024/wdoecm_Aug2024_v2.gdb/wdoecm_poly_point_basemap_intersect_statusyr2024_Aug2024_PA_DEF_moll'
oecm22 = 'O:/PP2024/wdoecm_Aug2024_v2.gdb/wdoecm_poly_point_basemap_intersect_statusyr2022_Aug2024_PA_DEF_moll'
oecm20 = 'O:/PP2024/wdoecm_Aug2024_v2.gdb/wdoecm_poly_point_basemap_intersect_statusyr2020_Aug2024_PA_DEF_moll'

wdpa_timeserise = [wdpa24, wdpa22, wdpa20, oecm24, oecm22, oecm20]

## count features in each file read in to make sure correct files are being used
for i in wdpa_timeserise:
    count = arcpy.management.GetCount(i)
    print(count)
    

649499
641030
628791
54799
52869
40837


In [4]:
## intersect wdpas with ecoregion

output_intersects = ['O:/PP2024/chapter5_final/ecoregions_terr/ecoregions_terr.gdb/wdpa24_ecoregion_intersect',
                    'O:/PP2024/chapter5_final/ecoregions_terr/ecoregions_terr.gdb/wdpa22_ecoregion_intersect',
                    'O:/PP2024/chapter5_final/ecoregions_terr/ecoregions_terr.gdb/wdpa20_ecoregion_intersect',
                     'O:/PP2024/chapter5_final/ecoregions_terr/ecoregions_terr.gdb/oecm24_ecoregion_intersect',
                     'O:/PP2024/chapter5_final/ecoregions_terr/ecoregions_terr.gdb/oecm22_ecoregion_intersect',
                     'O:/PP2024/chapter5_final/ecoregions_terr/ecoregions_terr.gdb/oecm20_ecoregion_intersect'
                    ]

for wdpa_timeserise, output_intersects in zip(wdpa_timeserise, output_intersects):
    input_features = [eco, wdpa_timeserise]
    eco_intersect = arcpy.analysis.PairwiseIntersect(input_features, output_intersects)
    
print("intersect complete")

intersect complete


In [5]:
## repair geometry of intersected layers 
wdpa24_intersect = 'O:/PP2024/chapter5_final/ecoregions_terr/ecoregions_terr.gdb/wdpa24_ecoregion_intersect'
wdpa22_intersect = 'O:/PP2024/chapter5_final/ecoregions_terr/ecoregions_terr.gdb/wdpa22_ecoregion_intersect'
wdpa20_intersect = 'O:/PP2024/chapter5_final/ecoregions_terr/ecoregions_terr.gdb/wdpa20_ecoregion_intersect'
oecm24_intersect = 'O:/PP2024/chapter5_final/ecoregions_terr/ecoregions_terr.gdb/oecm24_ecoregion_intersect'
oecm22_intersect = 'O:/PP2024/chapter5_final/ecoregions_terr/ecoregions_terr.gdb/oecm22_ecoregion_intersect'
oecm20_intersect = 'O:/PP2024/chapter5_final/ecoregions_terr/ecoregions_terr.gdb/oecm20_ecoregion_intersect'

eco_intersects = [wdpa24_intersect, wdpa22_intersect, wdpa20_intersect, oecm24_intersect, oecm22_intersect, oecm20_intersect]

for i in eco_intersects:
    arcpy.management.RepairGeometry(i, "DELETE_NULL", "OGC")
    print("repair complete")

repair complete
repair complete
repair complete
repair complete
repair complete
repair complete


In [6]:
## calculate area
for i in eco_intersects:
     arcpy.management.CalculateGeometryAttributes(
        in_features = i,
        geometry_property = [["area_km2", "AREA_GEODESIC"]],
        area_unit = "SQUARE_KILOMETERS",
        coordinate_system = 54009
     )

print("Geometry calculated")

Geometry calculated


In [7]:
## summary stats to calculate PA coverage of each ecoregion and export csv
output_tables = ['O:/PP2024/chapter5_final/ecoregions_terr/eco_terr_output_tables/wdpa24_ecoregion_intersect.csv',
                 'O:/PP2024/chapter5_final/ecoregions_terr/eco_terr_output_tables/wdpa22_ecoregion_intersect.csv',
                 'O:/PP2024/chapter5_final/ecoregions_terr/eco_terr_output_tables/wdpa20_ecoregion_intersect.csv',
                 'O:/PP2024/chapter5_final/ecoregions_terr/eco_terr_output_tables/oecm24_ecoregion_intersect.csv',
                 'O:/PP2024/chapter5_final/ecoregions_terr/eco_terr_output_tables/oecm22_ecoregion_intersect.csv',
                 'O:/PP2024/chapter5_final/ecoregions_terr/eco_terr_output_tables/oecm20_ecoregion_intersect.csv'
                ]

for eco_intersects, output_tables in zip(eco_intersects, output_tables):
    arcpy.analysis.Statistics(
        in_table = eco_intersects, 
        out_table = output_tables, 
        statistics_fields = [["area_km2", "SUM"]], 
        case_field = ["ECO_ID", "ECO_NAME", "BIOME_NAME", "REALM"]
    )

print("statistics calculated")

statistics calculated
